In [5]:
# 3.1 DATA SIMULATION (3 puntos) — Python
# ------------------------------------------------------------
# (2 pts) Simular n=1000 con X1..X4, D~Bernoulli(0.5), epsilon~N(0,1)
import numpy as np
import pandas as pd
from scipy import stats

rng = np.random.default_rng(123)
n = 1000

# Covariables: mezcla continuo/binario
x1 = rng.normal(0, 1, n)              # continuo
x2 = rng.normal(2, 1, n)              # continuo
x3 = rng.binomial(1, 0.4, n)          # binario
x4 = rng.uniform(-1, 1, n)            # continuo

# Asignación tratamiento
D  = rng.binomial(1, 0.5, n)

# Error y outcome: Y = 2D + 0.5X1 − 0.3X2 + 0.2X3 + ε, ε~N(0,1)
eps = rng.normal(0, 1, n)
Y = 2*D + 0.5*x1 - 0.3*x2 + 0.2*x3 + eps

# Guardar todo en un DataFrame
df = pd.DataFrame({
    "Y": Y, "D": D, "X1": x1, "X2": x2, "X3": x3, "X4": x4
})

# (sanidad opcional) verificar NA
assert df.isna().sum().sum() == 0, "Hay NA en el DataFrame"

# ------------------------------------------------------------
# (1 pt) Balance check: comparar medias de X1..X4 entre D=1 y D=0 con t-test
covs = ["X1", "X2", "X3", "X4"]

# Medias por grupo
group_means = df.groupby("D")[covs].mean().rename(index={0:"Control",1:"Tratado"})

# Welch t-tests y tabla resumen
rows = []
for c in covs:
    a = df.loc[df["D"]==1, c]
    b = df.loc[df["D"]==0, c]
    t_stat, p_val = stats.ttest_ind(a, b, equal_var=False)
    rows.append({
        "Covariable": c,
        "Media Tratado": a.mean(),
        "Media Control": b.mean(),
        "Diferencia": a.mean() - b.mean(),
        "t": t_stat,
        "p_Valor": p_val
    })
balance = pd.DataFrame(rows)

# Salidas principales
print("== Medias por grupo ==")
print(group_means.round(3))
print("\n== Balance por t-test (Welch) ==")
print(balance.round(4))


== Medias por grupo ==
            X1     X2     X3     X4
D                                  
Control  0.039  1.996  0.380 -0.005
Tratado  0.005  1.993  0.388  0.012

== Balance por t-test (Welch) ==
  Covariable  Media Tratado  Media Control  Diferencia       t  p_Valor
0         X1         0.0050         0.0391     -0.0341 -0.5359   0.5921
1         X2         1.9934         1.9965     -0.0031 -0.0472   0.9624
2         X3         0.3884         0.3795      0.0089  0.2900   0.7719
3         X4         0.0120        -0.0047      0.0166  0.4651   0.6419


In [6]:
# 3.2 ESTIMATING THE AVERAGE TREATMENT EFFECT (3 puntos) — Python
# Requiere 'df' del Paso 3.1: columnas Y, D, X1..X4.
# Si no existe, se re-crea con la misma semilla para reproducibilidad.
import numpy as np, pandas as pd
import statsmodels.api as sm

if 'df' not in globals():
    rng = np.random.default_rng(123); n=1000
    x1 = rng.normal(0,1,n); x2 = rng.normal(2,1,n); x3 = rng.binomial(1,0.4,n); x4 = rng.uniform(-1,1,n)
    D  = rng.binomial(1,0.5,n); eps = rng.normal(0,1,n)
    Y = 2*D + 0.5*x1 - 0.3*x2 + 0.2*x3 + eps
    df = pd.DataFrame({"Y":Y,"D":D,"X1":x1,"X2":x2,"X3":x3,"X4":x4})

def fit_ols(y, X, robust=True):
    Xc = sm.add_constant(X, has_constant='add')
    model = sm.OLS(y, Xc).fit(cov_type='HC1' if robust else 'nonrobust')
    return model

# (1 pt) ATE simple: Y ~ D
m_simple = fit_ols(df['Y'], df[['D']], robust=True)
ate_simple = m_simple.params['D']
se_simple  = m_simple.bse['D']
ci_simple  = m_simple.conf_int().loc['D'].tolist()
p_simple   = m_simple.pvalues['D']

print("== 3.2.1 ATE simple: Y ~ D ==")
print(f"ATE (coef D): {ate_simple:.4f}  SE: {se_simple:.4f}  95% CI: [{ci_simple[0]:.4f}, {ci_simple[1]:.4f}]  p={p_simple:.4g}")

# (1 pt) ATE con controles: Y ~ D + X1 + X2 + X3 + X4
m_ctrl = fit_ols(df['Y'], df[['D','X1','X2','X3','X4']], robust=True)
ate_ctrl = m_ctrl.params['D']
se_ctrl  = m_ctrl.bse['D']
ci_ctrl  = m_ctrl.conf_int().loc['D'].tolist()
p_ctrl   = m_ctrl.pvalues['D']

print("\n== 3.2.2 ATE con controles: Y ~ D + X1 + X2 + X3 + X4 ==")
print(f"ATE (coef D): {ate_ctrl:.4f}  SE: {se_ctrl:.4f}  95% CI: [{ci_ctrl[0]:.4f}, {ci_ctrl[1]:.4f}]  p={p_ctrl:.4g}")

# (1 pt) 3.2.3 Comparación: ¿cambia el ATE? ¿qué pasa con los SE?
delta_ate = ate_ctrl - ate_simple
se_change = se_ctrl - se_simple
ratio_se  = se_ctrl / se_simple if se_simple!=0 else np.nan

print("\n== 3.2.3 Comparación de estimaciones ==")
print(f"Cambio en ATE (controles - simple): {delta_ate:.4f}")
print(f"Cambio en SE: {se_change:.4f}   Ratio SE (ctrl/simple): {ratio_se:.3f}")
print("Nota: En un RCT bien balanceado, el ATE debería mantenerse cerca y los SE suelen bajar al agregar controles predictivos de Y.")


== 3.2.1 ATE simple: Y ~ D ==
ATE (coef D): 2.0527  SE: 0.0712  95% CI: [1.9131, 2.1924]  p=1.521e-182

== 3.2.2 ATE con controles: Y ~ D + X1 + X2 + X3 + X4 ==
ATE (coef D): 2.0633  SE: 0.0625  95% CI: [1.9409, 2.1857]  p=2.562e-239

== 3.2.3 Comparación de estimaciones ==
Cambio en ATE (controles - simple): 0.0106
Cambio en SE: -0.0088   Ratio SE (ctrl/simple): 0.877
Nota: En un RCT bien balanceado, el ATE debería mantenerse cerca y los SE suelen bajar al agregar controles predictivos de Y.


In [7]:
# 3.3 LASSO AND VARIABLE SELECTION (3 puntos) — Python
# Equivalente a cv.glmnet en R usando LassoCV de scikit-learn
import numpy as np, pandas as pd
from sklearn.linear_model import LassoCV
import statsmodels.api as sm

# ------------------------------------------------------------
# (1 pt) Ajustar LASSO de Y ~ X1..X4, EXCLUYENDO D
X = df[['X1','X2','X3','X4']].to_numpy()
y = df['Y'].to_numpy()

# Validación cruzada (10 folds por defecto). Normalizamos internamente.
lasso = LassoCV(cv=10, random_state=123).fit(X, y)

coef = pd.Series(lasso.coef_, index=['X1','X2','X3','X4'])
selected = coef[coef != 0].index.tolist()

print("== 3.3.1 LASSO SELECCIÓN ==")
print("Alpha (λ) óptimo:", lasso.alpha_)
print("Coeficientes estimados:")
print(coef.round(4))
print("Covariables seleccionadas en λ_min:", selected)

# ------------------------------------------------------------
# (1 pt) Re-estimar ATE con covariables seleccionadas
if selected:
    X_sel = df[['D'] + selected]
else:
    X_sel = df[['D']]

Xc = sm.add_constant(X_sel, has_constant='add')
m_lasso = sm.OLS(df['Y'], Xc).fit(cov_type='HC1')
ate_lasso = m_lasso.params['D']
se_lasso  = m_lasso.bse['D']
ci_lasso  = m_lasso.conf_int().loc['D'].tolist()

print("\n== 3.3.2 ATE con covariables seleccionadas por LASSO ==")
print(f"ATE (coef D): {ate_lasso:.4f}  SE: {se_lasso:.4f}  95% CI: [{ci_lasso[0]:.4f}, {ci_lasso[1]:.4f}]")

# ------------------------------------------------------------
# (1 pt) Comparación con 3.2
# (suponiendo que m_simple y m_ctrl ya fueron calculados)
print("\n== 3.3.3 Comparación ==")
print(f"ATE simple (3.2.1): {m_simple.params['D']:.4f}")
print(f"ATE con todos los controles (3.2.2): {m_ctrl.params['D']:.4f}")
print(f"ATE con controles seleccionados por LASSO (3.3.2): {ate_lasso:.4f}")
print("Comentario: El ATE debería ser estable porque el tratamiento fue asignado aleatoriamente.")
print("El uso de LASSO puede reducir ruido, mejorar precisión y seleccionar solo covariables relevantes.")


== 3.3.1 LASSO SELECCIÓN ==
Alpha (λ) óptimo: 0.0007311184038558634
Coeficientes estimados:
X1    0.4201
X2   -0.3031
X3    0.2599
X4    0.1007
dtype: float64
Covariables seleccionadas en λ_min: ['X1', 'X2', 'X3', 'X4']

== 3.3.2 ATE con covariables seleccionadas por LASSO ==
ATE (coef D): 2.0633  SE: 0.0625  95% CI: [1.9409, 2.1857]

== 3.3.3 Comparación ==
ATE simple (3.2.1): 2.0527
ATE con todos los controles (3.2.2): 2.0633
ATE con controles seleccionados por LASSO (3.3.2): 2.0633
Comentario: El ATE debería ser estable porque el tratamiento fue asignado aleatoriamente.
El uso de LASSO puede reducir ruido, mejorar precisión y seleccionar solo covariables relevantes.
